# Setup Gym Environment

```py
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.05 # when to give up
my_config["start_obs_capture"] = 0.05 # when to capture 
my_config["time_step_timeout_factor"] = 1.0 # how late is OK
my_config["act_buf_len"] = 3 # how many past actions
my_config["reset_act_buf"] = True # resect action buffer on reset
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2
```

This section needs to be setup for any method

In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [2]:
debugAsGym = False
testResult = False

from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import gymnasium

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.1
my_config["start_obs_capture"] = 0.1
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 224
my_config["act_buf_len"] = 3
my_config["reset_act_buf"] = False
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2

my_config["interface_kwargs"] = {
  'debugFlag': False, # do not use render() while True
  'img_hist_len': 3,
  'modelMode': 4,
  'agent' : 'SAC',
  #  [42, 42, K], [84, 84, K], [10, 10, K], [240, 320, K] and  [480, 640, K]
  'imageWidth' : 42, # there is a default Cov layer for PPO with 240 x 320
  'imageHeight' : 42,
  'trackChoice' : 3, # 1 is High Speed Ring, 2 is 0-400m in MR2, #3 is 0-400m in Supra, #4 is test ring
}

In [3]:
if debugAsGym:
    env = gymnasium.make("real-time-gym-v1", config=my_config)

In [4]:
if debugAsGym:
    env.reset()

# Register the environment in a way that RLlib is happy

In [5]:
if not debugAsGym and not testResult:
    def env_creator(env_config):
        env = gymnasium.make("real-time-gym-v1", config=env_config)
        return env  # return an env instance

    from ray.tune.registry import register_env
    register_env("gt-rtgym-env-v1", lambda config: env_creator(my_config)) # better way

In [6]:
if not debugAsGym and not testResult:
    import ray
    ray.shutdown()
    ray.init()

2023-09-25 22:43:24,115	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


In [8]:
if not debugAsGym and not testResult:
    from ray.rllib.algorithms.sac import SACConfig
    from ray.rllib.algorithms.ppo import PPOConfig
    from ray.rllib.algorithms.a2c import A2CConfig

    algo = (
        SACConfig()
        .resources(
            num_gpus=1,
            num_gpus_per_learner_worker=1,
            )
        .rollouts(
            #num_rollout_workers=1,
            enable_connectors=True,
            batch_mode="truncate_episodes",
            #num_envs_per_worker=1
            #rollout_fragment_length=256
            )
        .framework(
            framework="tf2",
            #eager_tracing=True,
            )
        .environment(
            env="gt-rtgym-env-v1",
            disable_env_checking=True,
            render_env=False,
            )
        .training(
            #lr=tune.grid_search([0.01, 0.001, 0.0001])
            #lambda_=0.95,
            #gamma=0.99,
            #sgd_minibatch_size=128,
            train_batch_size=155,
            #num_sgd_iter=8,
            #clip_param=0.2,
            #model={"fcnet_hiddens": [1, 8]},
        )
        .build()
    )

c:\Users\nadir\anaconda3\envs\GTRay2\lib\site-packages\ray\rllib\algorithms\algorithm.py:484: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
c:\Users\nadir\anaconda3\envs\GTRay2\lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
c:\Users\nadir\anaconda3\envs\GTRay2\lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be rem

GT Real Time instantiated
GT AI Server instantiated for rtgym
still simple reward system
starting up on localhost port 9999
Waiting for a connection


2023-09-25 22:43:56,094	WARNING deprecation.py:50 -- DeprecationWarning: `_build_eager_tf_policy` has been deprecated. This will raise an error in the future!
2023-09-25 22:43:56,094	WARNING deprecation.py:50 -- DeprecationWarning: `EagerTFPolicy` has been deprecated. This will raise an error in the future!


Connection from ('127.0.0.1', 55554)


2023-09-25 22:43:56,539	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.tf.tf_modelv2.TFModelV2` has been deprecated. Use `ray.rllib.core.rl_module.RLModule` instead. This will raise an error in the future!
2023-09-25 22:43:56,539	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.tf.fcnet.FullyConnectedNetwork` has been deprecated. This will raise an error in the future!
2023-09-25 22:43:56,547	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.tf.misc.normc_initializer` has been deprecated. This will raise an error in the future!
2023-09-25 22:43:56,595	WARNING deprecation.py:50 -- DeprecationWarning: `rllib.models.tf.ComplexInputNetwork` has been deprecated. This will raise an error in the future!
2023-09-25 22:43:59,835	WARNING deprecation.py:50 -- DeprecationWarning: `StochasticSampling` has been deprecated. This will raise an error in the future!
2023-09-25 22:43:59,835	WARNING deprecation.py:50 -- DeprecationWarning: `Exploratio

AssertionError: 

In [ ]:
if not debugAsGym and not testResult:
    N = 1000

    for n in range(N):
        result = algo.train()
        print("Loop: ", n)
        if n % 10 == 0:
            print("Saved", n)
            algo.save()
            
    algo.save()


In [ ]:
if not debugAsGym and not testResult:
    N = 1000

    for n in range(N):
        result = algo.train()
        print("Loop: ", n)
        if n % 50 == 0:
            print("Saved", n)
            algo.save()
            
    algo.save()

In [ ]:
from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import gymnasium

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.1
my_config["start_obs_capture"] = 0.1
my_config["time_step_timeout_factor"] = 1.0
#my_config["ep_max_length"] = 224
my_config["act_buf_len"] = 3
my_config["reset_act_buf"] = False
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2

my_config["interface_kwargs"] = {
  'debugFlag': False, # do not use render() while True
  'img_hist_len': 3,
  'modelMode': 4,
  'agent' : 'PPO',
  #  [42, 42, K], [84, 84, K], [10, 10, K], [240, 320, K] and  [480, 640, K]
  'imageWidth' : 42, # there is a default Cov layer for PPO with 240 x 320
  'imageHeight' : 42,
  'trackChoice' : 3, # 1 is High Speed Ring, 2 is 0-400m, 
}

In [ ]:
if not debugAsGym and testResult:
    def env_creator(env_config):
        env = gymnasium.make("real-time-gym-v1", config=env_config)
        return env  # return an env instance

    from ray.tune.registry import register_env
    register_env("gt-rtgym-env-v1", lambda config: env_creator(my_config)) 

In [ ]:
if not debugAsGym and testResult:
    from ray.rllib.algorithms.algorithm import Algorithm
    algo = Algorithm.from_checkpoint("C:/Users/nadir/ray_results/PPO_gt-rtgym-env-v1_2023-05-19_07-37-37z3d6v2w2/checkpoint_000061")
    #algo = Algorithm.from_checkpoint("C:/Users/nadir/ray_results/PPO_gt-rtgym-env-v1_2023-05-19_07-37-37z3d6v2w2/checkpoint_002000")

In [ ]:
if not debugAsGym and testResult:
    result = algo.train()

In [ ]:
if not debugAsGym and testResult:

    policy = algo.get_policy()
    #print(policy.model)
    model = policy.model
    print(model)